In [ ]:
# robot_interface.py
#!/usr/bin/env python3
import rclpy
from rclpy.node import Node

from geometry_msgs.msg import Pose

class RobotInterface(Node):
    def __init__(self):
        super().__init__('robot_interface')

        # === MoveIt 그룹 설정 ===
        self.arm = MoveGroupCommander("manipulator")
        self.gripper = MoveGroupCommander("gripper")

        self.get_logger().info("✅ RobotInterface initialized (arm + gripper ready)")

    # 1️⃣ 목표 좌표 이동
    def move_to_pose(self, target_pose: Pose):
        """
        EE를 특정 좌표(x, y, z, orientation)로 이동
        :param target_pose: geometry_msgs.msg.Pose 타입
        """
        self.get_logger().info("Moving to target pose...")
        self.arm.set_pose_target(target_pose)
        self.arm.go(wait=True)
        self.arm.stop()
        self.arm.clear_pose_targets()
        self.get_logger().info("✅ Move complete")

    # 2️⃣ 그리퍼 open
    def open_gripper(self):
        self.get_logger().info("Opening gripper...")
        self.gripper.set_named_target("open")
        self.gripper.go(wait=True)
        self.get_logger().info("✅ Gripper opened")

    # 3️⃣ 그리퍼 close
    def close_gripper(self):
        self.get_logger().info("Closing gripper...")
        self.gripper.set_named_target("close")
        self.gripper.go(wait=True)
        self.get_logger().info("✅ Gripper closed")

    # 4️⃣ 홈포즈 이동
    def move_home(self):
        """MoveIt의 'home' 포즈로 복귀"""
        self.get_logger().info("Moving to home position...")
        self.arm.set_named_target("home")
        self.arm.go(wait=True)
        self.arm.stop()
        self.get_logger().info("✅ Home position reached")

    # 5️⃣ 원하는 포즈 이동 (예: 특정 이름으로 등록된 pose)
    def move_named_pose(self, pose_name: str):
        """
        MoveIt setup에서 미리 정의된 pose 이름으로 이동
        ex) 'vertical', 'ready', 'inspection' 등
        """
        self.get_logger().info(f"Moving to named pose: {pose_name}")
        self.arm.set_named_target(pose_name)
        self.arm.go(wait=True)
        self.arm.stop()
        self.get_logger().info(f"✅ Pose '{pose_name}' reached")

    # 6️⃣ 비상정지
    def emergency_stop(self):
        """
        현재 동작 즉시 정지
        """
        self.get_logger().warn("⚠️ Emergency stop triggered!")
        self.arm.stop()
        self.gripper.stop()

    # === 프로그램 단독 실행 시 테스트용 ===
    def test_sequence(self):
        self.move_home()
        self.open_gripper()
        self.close_gripper()
        self.move_named_pose("ready")

def main(args=None):
    rclpy.init(args=args)
    node = RobotInterface()

    # 단독 실행 테스트
    node.test_sequence()

    node.destroy_node()
    rclpy.shutdown()

if __name__ == "__main__":
    main()
